# Version avec 1 seul Curseur :

In [1]:
from PyQt5 import QtCore, QtGui, QtWidgets

import pyvisa as visa
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import sys
from datetime import datetime
import time
from time import sleep
import pandas as pd

import serial
import serial.tools.list_ports

#------
# %matplotlib qt
#------
matplotlib.use('Qt5Agg')
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg, NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure
#---

def Select_central_value(tab):      
    '''Fonction qui sélectionne l'élément central d'un liste ou d'un array'''
    
    N = len(tab)
    if N%2==0:
        return tab[ int( N/2 - 1 )  ]
    else:
        return tab[ int ( (N-1)/2 - 1 ) ]
    
def Get_closest_value_index(tab,value):
    '''Fonction qui renvoie l'indice de l element de tab dont la valeur est la plus proche de value'''
    index = Select_central_value( np.where( np.abs(tab-value) == np.min( np.abs(tab-value) ) )[0] )
    return index

def Carac_Spectre(spectre, tab_lambda):
    '''Fonction qui renvoie la largeur a mi hauteur et la longueur d onde du pic d un spectre'''
    S_max = np.max(spectre)
    indice_max = Get_closest_value_index(spectre, S_max)
    
    # On divise le spectre en 2 regions, une à gauche du pic (S_inf) et l'autre à droite (S_sup)
    S_sup = spectre[indice_max:]
    S_inf = spectre[:indice_max+1]
    
    
    # On recupere l'indice de intensite la plus proches de 1/2 S_max pour les 2 signaux S_sup et S_inf
    indice1 = Get_closest_value_index(S_inf,0.5*S_max  )
    indice2 = Get_closest_value_index(S_sup,0.5*S_max  ) + len(S_inf)
    
    # On calcule la largeur du spectre à mi-hauteur
    largeur_mi_hauteur = tab_lambda[indice2] - tab_lambda[indice1]
    
    # On recupere la longueur d'onde du pic du spectre
    lambda_laser = tab_lambda[indice_max]
    
    return largeur_mi_hauteur,lambda_laser




#------------------------------------------------------------------------------------------------------------------------

def Calcul_lambda(creneau, t, t_c, lambda_c):
    ''' Fonction qui a servie à déterminer la valeur de a = d \lambda / d t'''
    tab_indices = np.where( (creneau>=0.8) & (creneau<2) )[0]
    tab_differences= tab_indices[1:] - tab_indices[:-1]
    plt.figure()#
    plt.plot(creneau[tab_indices])  #
    plt.show()
    tab_indices = tab_indices[1:]
    tab_indices_vrais = tab_indices[tab_differences==1]
    indice_etalon = Select_central_value( tab_indices_vrais )
    t_etalon = t[indice_etalon]
    
    a = 1 / np.abs(t_c - t_etalon)
    print(a)#
    
    tab_lambda = tab_lambda = lambda_c + a * (t-t_c)
    return tab_lambda
    
#-----------------------------------------------------------------------------------------------


class MplCanvas(FigureCanvasQTAgg):
    
    def __init__(self, parent=None, width=5, height=4, dpi=100):
        fig = Figure(figsize=(width, height), dpi=dpi)
        self.axes = fig.add_subplot(111)
        super(MplCanvas, self).__init__(fig)

#------------------


#------------------ Fenêtre Paramètres oscilloscope

class Ui_Dialog(object):
    def setupUi(self, Dialog):
        Dialog.setObjectName("Dialog")
        Dialog.resize(428, 358)
        self.splitter_11 = QtWidgets.QSplitter(Dialog)
        self.splitter_11.setGeometry(QtCore.QRect(30, 20, 371, 311))
        self.splitter_11.setOrientation(QtCore.Qt.Vertical)
        self.splitter_11.setObjectName("splitter_11")
        self.splitter_10 = QtWidgets.QSplitter(self.splitter_11)
        self.splitter_10.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_10.setObjectName("splitter_10")
        self.label_params_oscillo = QtWidgets.QLabel(self.splitter_10)
        self.label_params_oscillo.setObjectName("label_params_oscillo")
        self.pushButton = QtWidgets.QPushButton(self.splitter_10)
        self.pushButton.setObjectName("pushButton")
        self.splitter = QtWidgets.QSplitter(self.splitter_11)
        self.splitter.setOrientation(QtCore.Qt.Horizontal)
        self.splitter.setObjectName("splitter")
        self.label_bd_rate = QtWidgets.QLabel(self.splitter)
        self.label_bd_rate.setObjectName("label_bd_rate")
        self.lineEdit_baud_rate = QtWidgets.QLineEdit(self.splitter)
        self.lineEdit_baud_rate.setObjectName("lineEdit_baud_rate")
        self.splitter_9 = QtWidgets.QSplitter(self.splitter_11)
        self.splitter_9.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_9.setObjectName("splitter_9")
        self.label_hscale = QtWidgets.QLabel(self.splitter_9)
        self.label_hscale.setObjectName("label_hscale")
        self.lineEdit_hscale = QtWidgets.QLineEdit(self.splitter_9)
        self.lineEdit_hscale.setObjectName("lineEdit_hscale")
        self.splitter_2 = QtWidgets.QSplitter(self.splitter_11)
        self.splitter_2.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_2.setObjectName("splitter_2")
        self.label_hoffset = QtWidgets.QLabel(self.splitter_2)
        self.label_hoffset.setObjectName("label_hoffset")
        self.lineEdit_hoffset = QtWidgets.QLineEdit(self.splitter_2)
        self.lineEdit_hoffset.setObjectName("lineEdit_hoffset")
        self.splitter_8 = QtWidgets.QSplitter(self.splitter_11)
        self.splitter_8.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_8.setObjectName("splitter_8")
        self.label_vscaleCH1 = QtWidgets.QLabel(self.splitter_8)
        self.label_vscaleCH1.setObjectName("label_vscaleCH1")
        self.lineEdit_vscaleCH1 = QtWidgets.QLineEdit(self.splitter_8)
        self.lineEdit_vscaleCH1.setObjectName("lineEdit_vscaleCH1")
        self.splitter_7 = QtWidgets.QSplitter(self.splitter_11)
        self.splitter_7.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_7.setObjectName("splitter_7")
        self.label_vscaleCH2 = QtWidgets.QLabel(self.splitter_7)
        self.label_vscaleCH2.setObjectName("label_vscaleCH2")
        self.lineEdit_vscaleCH2 = QtWidgets.QLineEdit(self.splitter_7)
        self.lineEdit_vscaleCH2.setObjectName("lineEdit_vscaleCH2")
        self.splitter_6 = QtWidgets.QSplitter(self.splitter_11)
        self.splitter_6.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_6.setObjectName("splitter_6")
        self.labe_offsetCH1_V = QtWidgets.QLabel(self.splitter_6)
        self.labe_offsetCH1_V.setObjectName("labe_offsetCH1_V")
        self.lineEdit_offsetCH1_V = QtWidgets.QLineEdit(self.splitter_6)
        self.lineEdit_offsetCH1_V.setObjectName("lineEdit_offsetCH1_V")
        self.splitter_5 = QtWidgets.QSplitter(self.splitter_11)
        self.splitter_5.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_5.setObjectName("splitter_5")
        self.label_offsetCH2_V = QtWidgets.QLabel(self.splitter_5)
        self.label_offsetCH2_V.setObjectName("label_offsetCH2_V")
        self.lineEdit_offsetCH2_V = QtWidgets.QLineEdit(self.splitter_5)
        self.lineEdit_offsetCH2_V.setObjectName("lineEdit_offsetCH2_V")
        self.splitter_4 = QtWidgets.QSplitter(self.splitter_11)
        self.splitter_4.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_4.setObjectName("splitter_4")
        self.label_offsetCH1_V = QtWidgets.QLabel(self.splitter_4)
        self.label_offsetCH1_V.setObjectName("label_offsetCH1_V")
        self.lineEdit_offsetCH1_div = QtWidgets.QLineEdit(self.splitter_4)
        self.lineEdit_offsetCH1_div.setObjectName("lineEdit_offsetCH1_div")
        self.splitter_3 = QtWidgets.QSplitter(self.splitter_11)
        self.splitter_3.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_3.setObjectName("splitter_3")
        self.label_offsetCH2_div = QtWidgets.QLabel(self.splitter_3)
        self.label_offsetCH2_div.setObjectName("label_offsetCH2_div")
        self.lineEdit_offsetCH2_div = QtWidgets.QLineEdit(self.splitter_3)
        self.lineEdit_offsetCH2_div.setObjectName("lineEdit_offsetCH2_div")

        self.retranslateUi(Dialog)
        QtCore.QMetaObject.connectSlotsByName(Dialog)

    def retranslateUi(self, Dialog):
        _translate = QtCore.QCoreApplication.translate
        Dialog.setWindowTitle(_translate("Dialog", "Paramètres de l\'oscilloscope"))
        self.label_params_oscillo.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:12pt; font-weight:600; text-decoration: underline;\">Paramètres Oscilloscope :</span></p></body></html>"))
        self.pushButton.setText(_translate("Dialog", "Valider les paramètres"))
        self.label_bd_rate.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:10pt;\">Baud rate :</span></p></body></html>"))
        self.label_hscale.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:10pt;\">Horizontal Scale (en s) :</span></p></body></html>"))
        self.label_hoffset.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:10pt;\">Horizontal offset (en s) :</span></p></body></html>"))
        self.label_vscaleCH1.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:10pt;\">Vertical Scale CH1 (en V) :</span></p></body></html>"))
        self.label_vscaleCH2.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:10pt;\">Vertical Scale CH2 (en V) :</span></p></body></html>"))
        self.labe_offsetCH1_V.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:10pt;\">Offset CH1 (en V) :</span></p></body></html>"))
        self.label_offsetCH2_V.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:10pt;\">Offset CH2 (en V) :</span></p></body></html>"))
        self.label_offsetCH1_V.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:10pt;\">Offset CH1 (en div) :</span></p></body></html>"))
        self.label_offsetCH2_div.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:10pt;\">Offset CH2 (en div) :</span></p></body></html>"))


#---------------------


class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        
        
        
        #--------------------------
        
        self.rm = visa.ResourceManager()    # Appareils Pyvisa disponibles 
        self.serial_port = serial.tools.list_ports.comports()   # Ports série disponibles
        
        #-------------- MainWindow
        MainWindow.setObjectName("MainWindow")
        #MainWindow.resize(646, 626)
        MainWindow.resize(646, 275)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        
        MainWindow.setStatusTip("Bienvenue, veuillez sélectionner l'oscilloscope")
        
        #-------------------- Plot Widget
        self.widget = QtWidgets.QWidget(self.centralwidget)
        self.widget.setGeometry(QtCore.QRect(10, 230, 621, 351))
        self.widget.setObjectName("widget")
        self.widget.hide() ###---------------
        #-------------
        self.splitter_7 = QtWidgets.QSplitter(self.centralwidget)
        self.splitter_7.setGeometry(QtCore.QRect(10, 8, 621, 211))
        self.splitter_7.setOrientation(QtCore.Qt.Vertical)
        self.splitter_7.setObjectName("splitter_7")
        self.splitter = QtWidgets.QSplitter(self.splitter_7)
        self.splitter.setOrientation(QtCore.Qt.Horizontal)
        self.splitter.setObjectName("splitter")
        self.label_connexion_type = QtWidgets.QLabel(self.splitter)
        self.label_connexion_type.setObjectName("label_connexion_type")
        self.label_bdrate = QtWidgets.QLabel(self.splitter)
        self.label_bdrate.setObjectName("label_bdrate")
        self.lineEdit_bdrate = QtWidgets.QLineEdit(self.splitter)
        self.lineEdit_bdrate.setObjectName("lineEdit_bdrate")
        self.label_timeout = QtWidgets.QLabel(self.splitter)
        self.label_timeout.setObjectName("label_timeout")
        self.lineEdit_timeout = QtWidgets.QLineEdit(self.splitter)
        self.lineEdit_timeout.setObjectName("lineEdit_timeout")
        self.splitter_2 = QtWidgets.QSplitter(self.splitter_7)
        self.splitter_2.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_2.setObjectName("splitter_2")
        
        #---------------- Selection oscilloscope--------------
        self.label_select_oscillo = QtWidgets.QLabel(self.splitter_2)
        self.label_select_oscillo.setObjectName("label_select_oscillo")
        self.comboBox_select_oscillo = QtWidgets.QComboBox(self.splitter_2)
        self.comboBox_select_oscillo.setObjectName("comboBox_select_oscillo")
        
        self.answer_IDN = []
        self.comboBox_select_oscillo.addItem('IDN des appareils connectés')
        
        self.nb_devices_pyvisa = len( self.rm.list_resources() )
        
        # On liste les appareils connectés avec Pyvisa
        for i in range( self.nb_devices_pyvisa ):   
            ressource = self.rm.open_resource( self.rm.list_resources()[i] ) 
            self.answer_IDN.append( ressource.query('*IDN?',0.001) )
            self.comboBox_select_oscillo.addItem( self.answer_IDN[i].strip())  # .strip() enlève le '\n'
            #ressource.close()
        
        self.list_ressources = list(np.copy(self.answer_IDN))
        
        # On liste les appareils connectés avec Pyserial
        for port in self.serial_port:
            #com_port = f"{port.device}"
            com_port = port.device
            ressource = serial.Serial(com_port, 9600, timeout=1)
            ressource.write(b'*IDN? \n')
            sleep(1)
            IDN = ressource.readline().decode('ascii')
            if IDN =='':
                self.answer_IDN.append( f"{port}"+'\n' )    # On vérifie s'il s'agit bien d'un oscillo Tektronix
                self.comboBox_select_oscillo.addItem( f"{port}" )
                self.list_ressources.append(com_port)
            if IDN!='': 
                self.answer_IDN.append( IDN )
                self.comboBox_select_oscillo.addItem( IDN.strip() )
                self.list_ressources.append(com_port)
            ressource.close()
        
        #-----------Params-------
        self.button_params_oscillo = QtWidgets.QPushButton(self.splitter_7)
        self.button_params_oscillo.setObjectName("button_params_oscillo")
        self.splitter_3 = QtWidgets.QSplitter(self.splitter_7)
        self.splitter_3.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_3.setObjectName("splitter_3")
        self.label_lambda_c = QtWidgets.QLabel(self.splitter_3)
        self.label_lambda_c.setObjectName("label_lambda_c")
        self.spinBox_lambda_c = QtWidgets.QSpinBox(self.splitter_3)
        self.spinBox_lambda_c.setMinimum(350)
        self.spinBox_lambda_c.setMaximum(1100)
        self.spinBox_lambda_c.setObjectName("spinBox_lambda_c")
        self.splitter_4 = QtWidgets.QSplitter(self.splitter_7)
        self.splitter_4.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_4.setObjectName("splitter_4")
        self.label_vcursor = QtWidgets.QLabel(self.splitter_4)
        self.label_vcursor.setObjectName("label_vcursor")
        self.button_appear_vcursor = QtWidgets.QPushButton(self.splitter_4)
        self.button_appear_vcursor.setObjectName("button_appear_vcursor")
        self.checkBox_cursor = QtWidgets.QPushButton(self.splitter_4)
        self.checkBox_cursor.setObjectName("checkBox_cursor")
        self.button_acquire = QtWidgets.QPushButton(self.splitter_7)
        self.button_acquire.setObjectName("button_acquire")
        self.splitter_5 = QtWidgets.QSplitter(self.splitter_7)
        self.splitter_5.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_5.setObjectName("splitter_5")
        self.label_lambda_pic = QtWidgets.QLabel(self.splitter_5)
        self.label_lambda_pic.setObjectName("label_lambda_pic")
        self.value_lambda_pic = QtWidgets.QLineEdit(self.splitter_5)
        self.value_lambda_pic.setReadOnly(True)
        self.value_lambda_pic.setObjectName("value_lambda_pic")
        self.label_largeur = QtWidgets.QLabel(self.splitter_5)
        self.label_largeur.setObjectName("label_largeur")
        self.label_largeur_mi_hauteur = QtWidgets.QLineEdit(self.splitter_5)
        self.label_largeur_mi_hauteur.setReadOnly(True)
        self.label_largeur_mi_hauteur.setObjectName("label_largeur_mi_hauteur")
        self.splitter_6 = QtWidgets.QSplitter(self.splitter_7)
        self.splitter_6.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_6.setObjectName("splitter_6")
        self.label_date_acquisition = QtWidgets.QLabel(self.splitter_6)
        self.label_date_acquisition.setObjectName("label_date_acquisition")
        self.label_date = QtWidgets.QLineEdit(self.splitter_6)
        self.label_date.setText("")
        self.label_date.setReadOnly(True)
        self.label_date.setObjectName("label_date")
        MainWindow.setCentralWidget(self.centralwidget)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)
        self.menuBar = QtWidgets.QMenuBar(MainWindow)
        self.menuBar.setGeometry(QtCore.QRect(0, 0, 646, 21))
        self.menuBar.setObjectName("menuBar")
        self.menuEnregister_l_acquisition_en = QtWidgets.QMenu(self.menuBar)
        self.menuEnregister_l_acquisition_en.setObjectName("menuEnregister_l_acquisition_en")
        #self.menuHelp = QtWidgets.QMenu(self.menuBar)
        #self.menuHelp.setObjectName("menuHelp")
        self.menuEnregistrer_l_acquisition_en = QtWidgets.QMenu(self.menuBar)
        self.menuEnregistrer_l_acquisition_en.setObjectName("menuEnregistrer_l_acquisition_en")
        MainWindow.setMenuBar(self.menuBar)
        self.action_png = QtWidgets.QAction(MainWindow)
        self.action_png.setObjectName("action_png")
        self.action_jpg = QtWidgets.QAction(MainWindow)
        self.action_jpg.setObjectName("action_jpg")
        self.action_csv = QtWidgets.QAction(MainWindow)
        self.action_csv.setObjectName("action_csv")
        self.action_txt = QtWidgets.QAction(MainWindow)
        self.action_txt.setObjectName("action_txt")
        self.action_png_2 = QtWidgets.QAction(MainWindow)
        self.action_png_2.setObjectName("action_png_2")
        self.action_jpg_2 = QtWidgets.QAction(MainWindow)
        self.action_jpg_2.setObjectName("action_jpg_2")
        self.action_csv_2 = QtWidgets.QAction(MainWindow)
        self.action_csv_2.setObjectName("action_csv_2")
        self.action_txt_2 = QtWidgets.QAction(MainWindow)
        self.action_txt_2.setObjectName("action_txt_2")
        self.action_png_3 = QtWidgets.QAction(MainWindow)
        self.action_png_3.setObjectName("action_png_3")
        self.action_jpg_3 = QtWidgets.QAction(MainWindow)
        self.action_jpg_3.setObjectName("action_jpg_3")
        self.action_csv_3 = QtWidgets.QAction(MainWindow)
        self.action_csv_3.setObjectName("action_csv_3")
        self.action_txt_3 = QtWidgets.QAction(MainWindow)
        self.action_txt_3.setObjectName("action_txt_3")
        self.actionGuide_d_utilisation = QtWidgets.QAction(MainWindow)
        self.actionGuide_d_utilisation.setObjectName("actionGuide_d_utilisation")
        self.action_csv_4 = QtWidgets.QAction(MainWindow)
        self.action_csv_4.setObjectName("action_csv_4")
        self.action_txt_4 = QtWidgets.QAction(MainWindow)
        self.action_txt_4.setObjectName("action_txt_4")
        self.menuEnregister_l_acquisition_en.addAction(self.action_png_3)
        self.menuEnregister_l_acquisition_en.addAction(self.action_jpg_3)
        #self.menuHelp.addAction(self.actionGuide_d_utilisation)
        self.menuEnregistrer_l_acquisition_en.addAction(self.action_csv_4)
        self.menuEnregistrer_l_acquisition_en.addAction(self.action_txt_4)
        self.menuBar.addAction(self.menuEnregistrer_l_acquisition_en.menuAction())
        self.menuBar.addAction(self.menuEnregister_l_acquisition_en.menuAction())
        #self.menuBar.addAction(self.menuHelp.menuAction())

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)
        
        
        # --------------- Figure-----------------------------------------------------------
    
    
    
        self.sc = MplCanvas(MainWindow, width=5, height=4, dpi=100)
        self.toolbar = NavigationToolbar(self.sc, MainWindow)
        self.layout = QtWidgets.QVBoxLayout()
        self.layout.addWidget(self.toolbar)
        self.layout.addWidget(self.sc)
        self.widget.setLayout(self.layout)
        
        
        #----- Fenêtre Paramètres Oscilloscope-----------------------------------------------------------
        self.Dialog = QtWidgets.QDialog()
        self.ui_Dialog = Ui_Dialog()
        self.ui_Dialog.setupUi(self.Dialog)
        self.ui_Dialog.lineEdit_offsetCH1_div.setReadOnly(True)
        self.ui_Dialog.lineEdit_offsetCH2_div.setReadOnly(True)

        # --------------------Actions--------------------
        
        # Selection de l'oscilloscope via la ComboBox
        self.comboBox_select_oscillo.activated[str].connect(self.select_ressource)
        
        # faire apparaître les curseurs verticaux
        self.button_appear_vcursor.clicked.connect(self.appear_vcursor)
        
        # Valider les positions des curseurs
        self.checkBox_cursor.clicked.connect(self.maj_cursors)
        
        # Acquisition
        self.button_acquire.clicked.connect(self.acquire_data)
        
        self.connect = None   # type de la connexion : pyvisa ou pyserial
        
        # Ouvrir la fenêtre paramètres de l'oscilloscope :
        self.button_params_oscillo.clicked.connect(self.open_params_window)
        
        self.ui_Dialog.pushButton.clicked.connect(self.update_params)
        
        
        
        #----------------------------------------- QActions
        
        self.action_csv_4.triggered.connect(self.save_acq_as_csv)
        self.action_txt_4.triggered.connect(self.save_acq_as_txt)
        
        self.action_png_3.triggered.connect(self.save_fig_as_png)
        self.action_jpg_3.triggered.connect(self.save_fig_as_jpg)
        
        
        
        
        
        
        #-----------------------------------------------------------

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.label_connexion_type.setText(_translate("MainWindow", "<html><head/><body><p><span style=\" font-size:10pt;\">Connexion avec pyserial :</span></p></body></html>"))
        self.label_bdrate.setText(_translate("MainWindow", "Baudrate"))
        self.label_timeout.setText(_translate("MainWindow", "Timeout"))
        self.label_select_oscillo.setText(_translate("MainWindow", "<html><head/><body><p><span style=\" font-size:10pt; font-weight:600;\">Selectionnez l\'oscilloscope :</span></p></body></html>"))
        self.button_params_oscillo.setText(_translate("MainWindow", "Paramètres de l\'oscilloscope"))
        self.label_lambda_c.setText(_translate("MainWindow", "<html><head/><body><p><span style=\" font-size:10pt;\">Veuillez entrer la longuer d\'onde associée au curseur de l\'analyseur de spectres (nm) :</span></p></body></html>"))
        self.label_vcursor.setText(_translate("MainWindow", "<html><head/><body><p><span style=\" font-size:9pt;\">Veuillez placer le curseur 1 de l\'oscilloscope sur le curseur de l\'analyseur :</span></p></body></html>"))
        self.button_appear_vcursor.setText(_translate("MainWindow", "Faire apparaître le curseur"))
        self.checkBox_cursor.setText(_translate("MainWindow", "Valider"))
        self.button_acquire.setText(_translate("MainWindow", "Lancer l\'acquisition"))
        self.label_lambda_pic.setText(_translate("MainWindow", "<html><head/><body><p><span style=\" font-size:10pt;\">Longueur d\'onde du pic :</span></p></body></html>"))
        self.label_largeur.setText(_translate("MainWindow", "<html><head/><body><p><span style=\" font-size:10pt;\">Largeur à mi-hauteur :</span></p></body></html>"))
        self.label_date_acquisition.setText(_translate("MainWindow", "<html><head/><body><p><span style=\" font-size:10pt;\">Date de l\'acquisition :</span></p></body></html>"))
        self.menuEnregister_l_acquisition_en.setTitle(_translate("MainWindow", "Enregister la figure en"))
        #self.menuHelp.setTitle(_translate("MainWindow", "Help"))
        self.menuEnregistrer_l_acquisition_en.setTitle(_translate("MainWindow", "Enregistrer l\'acquisition en"))
        self.action_png.setText(_translate("MainWindow", ".png"))
        self.action_jpg.setText(_translate("MainWindow", ".jpg"))
        self.action_csv.setText(_translate("MainWindow", ".csv"))
        self.action_txt.setText(_translate("MainWindow", ".txt"))
        self.action_png_2.setText(_translate("MainWindow", ".png"))
        self.action_jpg_2.setText(_translate("MainWindow", ".jpg"))
        self.action_csv_2.setText(_translate("MainWindow", ".csv"))
        self.action_txt_2.setText(_translate("MainWindow", ".txt"))
        self.action_png_3.setText(_translate("MainWindow", ".png"))
        self.action_jpg_3.setText(_translate("MainWindow", ".jpg"))
        self.action_csv_3.setText(_translate("MainWindow", ".csv"))
        self.action_txt_3.setText(_translate("MainWindow", ".txt"))
        self.actionGuide_d_utilisation.setText(_translate("MainWindow", "Guide d\'utilisation"))
        self.action_csv_4.setText(_translate("MainWindow", ".csv"))
        self.action_txt_4.setText(_translate("MainWindow", ".txt"))
        
    #   ----------------------------- Fonctions ----- Logique -----------------------------
    
        
    def select_ressource(self, text):    # Selection de l'appareil via la ComboBox
        try:
            self.index_oscillo = int(np.where( np.array(self.answer_IDN)==text+'\n' )[0])
            if self.index_oscillo < self.nb_devices_pyvisa:
                self.oscillo = self.rm.open_resource( self.rm.list_resources()[self.index_oscillo] )
                print("Appareil connecté avec Pyvisa")  ###
                print("IDN de le l'appareil sélectionné :"+self.oscillo.query('*IDN?',0.001)) ###
                
                MainWindow.setStatusTip("Connexion avec Pyvisa")
                
                self.connect = 'pyvisa'
                                
                # UPDATE PARAMS OSCILLO
                
                self.ui_Dialog.lineEdit_baud_rate.setText(self.oscillo.query('RS232:BAUD?',0.01))
                echelle1 = float(self.oscillo.query('CH1:SCALE?',0.01))
                offset1_div = float(self.oscillo.query('CH1:POS?',0.01))
                offset1_V = offset1_div * echelle1
                self.ui_Dialog.lineEdit_vscaleCH1.setText(str(echelle1))
                self.ui_Dialog.lineEdit_offsetCH1_V.setText(str(offset1_V))
                self.ui_Dialog.lineEdit_offsetCH1_div.setText(str(offset1_div))
                echelle2 = float(self.oscillo.query('CH2:SCALE?',0.01))
                offset2_div = float(self.oscillo.query('CH2:POS?',0.01))
                offset2_V = offset2_div * echelle2
                self.ui_Dialog.lineEdit_vscaleCH2.setText(str(echelle2))
                self.ui_Dialog.lineEdit_offsetCH2_V.setText(str(offset2_V))
                self.ui_Dialog.lineEdit_offsetCH2_div.setText(str(offset2_div))
                self.ui_Dialog.lineEdit_hscale.setText(self.oscillo.query('HORIZONTAL:SCALE?',0.01))
                self.ui_Dialog.lineEdit_hoffset.setText(self.oscillo.query('HORIZONTAL:DELAY:POS?',0.01))
                
            else:
                self.bd_rate = int(self.lineEdit_bdrate.text())
                self.timeout = int(self.lineEdit_timeout.text())
                self.com = self.list_ressources[self.index_oscillo]
                self.oscillo = serial.Serial(self.com, self.bd_rate, timeout=self.timeout)
                print("Appareil connecté avec Pyserial")   ###
                print("IDN de le l'appareil sélectionné :"+self.answer_IDN[self.index_oscillo]) ###
                
                MainWindow.setStatusTip("Connexion avec Pyserial "+self.com)
                
                self.connect = 'pyserial'
                
                #---- M A J params oscilloscope
                self.oscillo.write(b'RS232:BAUD? \n')
                self.ui_Dialog.lineEdit_baud_rate.setText(self.oscillo.readline().decode('ascii'))
                
                self.oscillo.write(b'CH1:SCALE? \n')
                echelle1 = float(self.oscillo.readline().decode('ascii'))
                self.oscillo.write(b'CH1:POS? \n')
                offset1_div = float(self.oscillo.readline().decode('ascii'))
                offset1_V = offset1_div * echelle1
                self.ui_Dialog.lineEdit_vscaleCH1.setText(str(echelle1))
                self.ui_Dialog.lineEdit_offsetCH1_V.setText(str(offset1_V))
                self.ui_Dialog.lineEdit_offsetCH1_div.setText(str(offset1_div))
                
                self.oscillo.write(b'CH2:SCALE? \n')
                echelle2 = float(self.oscillo.readline().decode('ascii'))
                self.oscillo.write(b'CH2:POS? \n')
                offset2_div = float(self.oscillo.readline().decode('ascii'))
                offset2_V = offset2_div * echelle2
                self.ui_Dialog.lineEdit_vscaleCH2.setText(str(echelle2))
                self.ui_Dialog.lineEdit_offsetCH2_V.setText(str(offset2_V))
                self.ui_Dialog.lineEdit_offsetCH2_div.setText(str(offset2_div))
                
                self.oscillo.write(b'HORIZONTAL:SCALE? \n')
                self.ui_Dialog.lineEdit_hscale.setText(self.oscillo.readline().decode('ascii'))
                
                self.oscillo.write(b'HORIZONTAL:DELAY:POS? \n')
                self.ui_Dialog.lineEdit_hoffset.setText(self.oscillo.readline().decode('ascii'))
                
        except:
            print("Pas d'appareil connecté, vérifier les paramètres entrés ou le branchement...")
            MainWindow.setStatusTip("Pas d'appareil connecté, vérifier les paramètres entrés ou le branchement...")
            
    #---------
        
    def appear_vcursor(self):
        if self.connect == "pyvisa":
            self.oscillo.write('CURSOR:FUNC VBA')
        elif self.connect == "pyserial":
            self.oscillo.write(b'CURSOR:FUNC VBA \n')
        else:
            print("Vous n'avez pas selectionné d'oscilloscope")
            MainWindow.setStatusTip("Vous n'avez pas selectionné d'oscilloscope")
        
    #---------
        
        
    def maj_cursors(self):
        self.lambda_c = self.spinBox_lambda_c.value()
        if self.connect == 'pyvisa':
            self.t_c_read = float( self.oscillo.query('CURSOR:VBARS:POSITION1?') )
            self.oscillo.write('CURSOR:FUNC OFF')
            print("Longueur d'onde du curseur : "+str(self.lambda_c)+' nm')
            MainWindow.setStatusTip("Longueur d'onde du curseur : "+str(self.lambda_c)+' nm')
        elif self.connect == 'pyserial':
            self.oscillo.write(b'CURSOR:VBARS:POSITION1? \n')
            self.t_c_read = float( self.oscillo.readline().decode('ascii') )
            self.oscillo.write(b'CURSOR:FUNC OFF \n')
            print("Longueur d'onde du curseur : "+str(self.lambda_c)+' nm')
            MainWindow.setStatusTip("Longueur d'onde du curseur : "+str(self.lambda_c)+' nm')
        else:
            print("Vous n'avez pas selectionné d'oscilloscope")
            MainWindow.setStatusTip("Vous n'avez pas selectionné d'oscilloscope")
        
    #---------
        
        
    def acquire_data(self):
        
        try:
            if self.connect=='pyvisa':
                self.oscillo.write('DATA:SOURCE CH1')  # Select CH1
                self.coef_spectre = float(self.oscillo.query('WFMP:YMULT?',0.001))
                self.date_acquisition = datetime.now()
                self.spectre = np.fromstring(self.oscillo.query('CURVE?',0.001), sep=',') * self.coef_spectre

                self.oscillo.write('DATA:SOURCE CH2')
                self.coef_creneau = float(self.oscillo.query('WFMP:YMULT?',0.001))
                self.creneaux = np.fromstring(self.oscillo.query('CURVE?',0.001), sep=',') * self.coef_creneau

                self.t0 = float(self.oscillo.query('WFMP:XZERO?'))  # temps initial
                self.dt = float(self.oscillo.query('WFMP:XIN?'))   # intervalle d'échantillionage
                
                self.CH1_scale = float(self.oscillo.query('CH1:SCALE?',0.001))
                self.CH2_scale = float(self.oscillo.query('CH2:SCALE?',0.001))

                self.offset_spectre = float(self.oscillo.query('CH1:POS?',0.001)) * self.CH1_scale
                self.offset_creneau = float(self.oscillo.query('CH2:POS?',0.001)) * self.CH2_scale

            elif self.connect=='pyserial':
                
                self.oscillo.write(b'WFMP:XZERO? \n')
                self.t0 = float(self.oscillo.readline().decode('ascii'))

                self.oscillo.write(b'WFMP:XIN? \n')
                self.dt = float(self.oscillo.readline().decode('ascii'))

                self.oscillo.write(b'CH1:POS? \n')
                offset_spectre = float(self.oscillo.readline().decode('ascii'))
                self.oscillo.write(b'CH2:POS? \n')
                offset_creneau = float(self.oscillo.readline().decode('ascii'))

                self.oscillo.write(b'CH1:SCALE? \n')
                self.CH1_scale = float(self.oscillo.readline().decode('ascii'))
                self.offset_spectre = offset_spectre * self.CH1_scale
                self.oscillo.write(b'CH2:SCALE? \n')
                self.CH2_scale = float(self.oscillo.readline().decode('ascii'))
                self.offset_creneau = offset_creneau * self.CH2_scale

                self.oscillo.write(b'DATA:SOURCE CH1 \n')  # Select CH1

                self.date_acquisition = datetime.now()
                self.oscillo.write(b'CURVE? \n')
                sleep(1)
                V1 = np.fromstring(self.oscillo.readline().decode('ascii'), sep=',')
                self.oscillo.write(b'WFMP:YMULT? \n')
                self.coef_spectre = float(self.oscillo.readline().decode('ascii'))

                self.spectre = V1 * self.coef_spectre

                self.oscillo.write(b'DATA:SOURCE CH2 \n')
                self.oscillo.write(b'WFMP:YMULT? \n')
                self.coef_creneau = float(self.oscillo.readline().decode('ascii'))
                self.oscillo.write(b'CURVE? \n')
                sleep(1)
                V2 = np.fromstring(self.oscillo.readline().decode('ascii'), sep=',')

                self.creneaux = V2 * self.coef_creneau
                
                #----------------------------------------------


            self.spectre_calcul = self.spectre - self.offset_spectre
            self.creneaux_calcul = self.creneaux - self.offset_creneau

            self.t = np.linspace(self.t0, self.t0 + 2500*self.dt,2500, endpoint=True)

            self.indice_curseur = Get_closest_value_index(self.t,self.t_c_read)

            self.t_c = self.t[self.indice_curseur]

            
            # Conversion temps---longueur d'onde----------------------------------------
            # Calcul de a
            self.a = 13680  # 14320.916905444135
            #self.tab_lambda = Calcul_lambda(self.creneaux, self.t, self.t_c, self.lambda_c)

            
            self.tab_lambda = self.lambda_c + self.a * (self.t-self.t_c)
            
            #---------------------------------------------------------------------------

            
            self.largeur_mi_hauteur, self.lambda_pic = Carac_Spectre(self.spectre_calcul, self.tab_lambda)

            self.value_lambda_pic.setText('%.1f nm' %self.lambda_pic)
            self.label_largeur_mi_hauteur.setText('%.1f nm' %self.largeur_mi_hauteur)
            date_mesure = self.date_acquisition.strftime('%Y-%m-%d') + '                             ' + self.date_acquisition.strftime('%H') + 'h : '+ self.date_acquisition.strftime('%M') + 'min : ' + self.date_acquisition.strftime('%S') + 's'
            self.label_date.setText(date_mesure) 

            # Mise-à-jour du plot
            self.layout.removeWidget(self.sc)
            self.layout.removeWidget(self.toolbar)
            self.sc = MplCanvas(MainWindow, width=5, height=3)  # , -dpi=100)
            self.toolbar = NavigationToolbar(self.sc, MainWindow)
            coef_affichage = np.max(self.spectre) / np.max(self.creneaux)
            self.sc.axes.plot(self.tab_lambda, self.creneaux / self.CH2_scale ,alpha=0.5)
            self.sc.axes.plot(self.tab_lambda, self.spectre / self.CH1_scale ,label='Spectre',color='r')
            self.sc.axes.set_xlabel(r'Longueur d onde $\lambda \; \; $(nm)')
            self.sc.axes.set_ylabel(r'Signal (V)')
            self.sc.axes.legend()
            self.layout.addWidget(self.toolbar)
            self.layout.addWidget(self.sc)
            
            #self.widget.setLayout(layout)
            
            print('Acquisition réussie avec succès')
            MainWindow.setStatusTip("Acquisition réussie avec succès")
            
            MainWindow.resize(646, 626)
            self.widget.show()
        
        except:
            print("Vous n'avez pas selectionné d'oscilloscope")
            MainWindow.setStatusTip("Vous n'avez pas selectionné d'oscilloscope")
            
            
    def open_params_window(self):
        self.Dialog.show()
        
        
    def update_params(self):
        try:
            if self.connect=='pyserial':

                baud_rate = int(self.ui_Dialog.lineEdit_baud_rate.text())
                self.oscillo.write(b'RS232:BAUD %i \n'%baud_rate)

                hscale = float(self.ui_Dialog.lineEdit_hscale.text())
                self.oscillo.write(b'HORIZONTAL:SCALE %f \n'%hscale)

                hoffset = float(self.ui_Dialog.lineEdit_hoffset.text())
                self.oscillo.write(b'HORIZONTAL:POS %f \n'%hoffset)

                vscale1 = float(self.ui_Dialog.lineEdit_vscaleCH1.text())
                self.oscillo.write(b'CH1:SCALE %f \n'%vscale1)

                vscale2 = float(self.ui_Dialog.lineEdit_vscaleCH2.text())
                self.oscillo.write(b'CH2:SCALE %f \n'%vscale2)

                echelle1 = float(self.ui_Dialog.lineEdit_offsetCH1_V.text())
                voffset1 = echelle1/vscale1
                self.oscillo.write(b'CH1:POS %f \n'%voffset1)

                echelle2 = float(self.ui_Dialog.lineEdit_offsetCH2_V.text())
                voffset2 = echelle2/vscale2
                self.oscillo.write(b'CH2:POS %f \n'%voffset2)

                sleep(1)
                # Mise à jour de l'affichage :

                self.oscillo.write(b'RS232:BAUD?\n')
                sleep(0.2)
                self.ui_Dialog.lineEdit_baud_rate.setText(self.oscillo.readline().decode('ascii'))   
                self.oscillo.write(b'CH1:SCALE? \n')
                sleep(0.1)
                echelle1 = float(self.oscillo.readline().decode('ascii'))
                self.oscillo.write(b'CH1:POS? \n')
                sleep(0.1)
                offset1_div = float(self.oscillo.readline().decode('ascii'))
                offset1_V = offset1_div * echelle1
                self.ui_Dialog.lineEdit_vscaleCH1.setText(str(echelle1))
                self.ui_Dialog.lineEdit_offsetCH1_V.setText(str(offset1_V))
                self.ui_Dialog.lineEdit_offsetCH1_div.setText(str(offset1_div))   
                self.oscillo.write(b'CH2:SCALE? \n')
                sleep(0.1)
                echelle2 = float(self.oscillo.readline().decode('ascii'))
                self.oscillo.write(b'CH2:POS? \n')
                sleep(0.1)
                offset2_div = float(self.oscillo.readline().decode('ascii'))
                offset2_V = offset2_div * echelle2
                self.ui_Dialog.lineEdit_vscaleCH2.setText(str(echelle2))
                self.ui_Dialog.lineEdit_offsetCH2_V.setText(str(offset2_V))
                self.ui_Dialog.lineEdit_offsetCH2_div.setText(str(offset2_div))      
                self.oscillo.write(b'HORIZONTAL:SCALE? \n')
                sleep(0.1)
                self.ui_Dialog.lineEdit_hscale.setText(self.oscillo.readline().decode('ascii'))     
                self.oscillo.write(b'HORIZONTAL:POS? \n')
                sleep(0.1)
                self.ui_Dialog.lineEdit_hoffset.setText(self.oscillo.readline().decode('ascii'))

                print("Paramètres de l'oscilloscope mis-à-jour")
                MainWindow.setStatusTip("Paramètres de l'oscilloscope mis-à-jour")

            elif self.connect=='pyvisa':

                baud_rate = int(self.ui_Dialog.lineEdit_baud_rate.text())
                self.oscillo.write('RS232:BAUD %i'%baud_rate)

                hscale = float(self.ui_Dialog.lineEdit_hscale.text())
                self.oscillo.write('HORIZONTAL:SCALE %f'%hscale)

                hoffset = float(self.ui_Dialog.lineEdit_hoffset.text())
                self.oscillo.write('HORIZONTAL:POS %f'%hoffset)

                vscale1 = float(self.ui_Dialog.lineEdit_vscaleCH1.text())
                self.oscillo.write('CH1:SCALE %f'%vscale1)

                vscale2 = float(self.ui_Dialog.lineEdit_vscaleCH2.text())
                self.oscillo.write('CH2:SCALE %f'%vscale2)

                echelle1 = float(self.ui_Dialog.lineEdit_offsetCH1_V.text())
                voffset1 = echelle1/vscale1
                self.oscillo.write('CH1:POS %f'%voffset1)

                echelle2 = float(self.ui_Dialog.lineEdit_offsetCH2_V.text())
                voffset2 = echelle2/vscale2
                self.oscillo.write('CH2:POS %f'%voffset2)

                sleep(1)
                # Mise à jour de l'affichage :

                self.ui_Dialog.lineEdit_baud_rate.setText(self.oscillo.query('RS232:BAUD?',0.01)) 
                sleep(0.1)
                echelle1 = float(self.oscillo.query('CH1:SCALE?',0.01))
                offset1_div = float(self.oscillo.query('CH1:POS?',0.01))  
                offset1_V = offset1_div * echelle1
                self.ui_Dialog.lineEdit_vscaleCH1.setText(str(echelle1))
                self.ui_Dialog.lineEdit_offsetCH1_V.setText(str(offset1_V))
                self.ui_Dialog.lineEdit_offsetCH1_div.setText(str(offset1_div))   

                sleep(0.1)
                echelle2 = float(self.oscillo.query('CH2:SCALE?',0.01))
                offset2_div = float(self.oscillo.query('CH2:POS?',0.01))  
                offset2_V = offset2_div * echelle2
                self.ui_Dialog.lineEdit_vscaleCH2.setText(str(echelle2))
                self.ui_Dialog.lineEdit_offsetCH2_V.setText(str(offset2_V))
                self.ui_Dialog.lineEdit_offsetCH2_div.setText(str(offset2_div)) 

                sleep(0.1)
                self.ui_Dialog.lineEdit_hscale.setText(self.oscillo.query('HORIZONTAL:SCALE?',0.01))     
                sleep(0.1)
                self.ui_Dialog.lineEdit_hoffset.setText(self.oscillo.query('HORIZONTAL:POS?',0.01))

                print("Paramètres de l'oscilloscope mis-à-jour")
                MainWindow.setStatusTip("Paramètres de l'oscilloscope mis-à-jour")

            else:
                print("Vous n'avez pas selectionné d'oscilloscope")
                MainWindow.setStatusTip("Vous n'avez pas selectionné d'oscilloscope")
        except:
            pass
        
        
    def save_acq_as_txt(self):
        try:
            N = self.tab_lambda.size
            tab_data = np.zeros(N, dtype=[('lambda (nm)',float),('spectre (V)',float),('creneau',float)])
            tab_data['lambda (nm)'] = self.tab_lambda
            tab_data['spectre (V)'] = self.spectre
            tab_data['creneau'] = self.creneaux

            np.savetxt('Acquisition.txt',tab_data, delimiter=',', fmt='%f')

            print("Acquisition enregistrée dans Acquisition.txt")
            MainWindow.setStatusTip("Acquisition enregistrée dans Acquisition.txt")
        except:
            print('Aucune acquisition réalisée')
            MainWindow.setStatusTip('Aucune acquisition réalisée')
        
    def save_acq_as_csv(self):
        try:
            tab_CH1_scale = self.CH1_scale * np.ones(len(self.spectre))
            tab_CH2_scale = self.CH2_scale * np.ones(len(self.spectre))
            dict_data = {"Longueur d'onde (nm)":self.tab_lambda, "Spectre (V)":self.spectre, "Créneau (V)":self.creneaux,"CH1 Scale (Volt/div)":tab_CH1_scale,"CH2 Scale (Volt/div)":tab_CH2_scale}
            data_frame = pd.DataFrame(data=dict_data)
            data_frame.to_csv("Acquisition.csv")

            print("Acquisition enregistrée dans Acquisition.csv")
            MainWindow.setStatusTip("Acquisition enregistrée dans Acquisition.csv")
        except:
            print('Aucune acquisition réalisée')
            MainWindow.setStatusTip('Aucune acquisition réalisée')
        
    def save_fig_as_png(self):
        try:
            figure = plt.figure()
            plt.plot(self.tab_lambda, self.spectre / self.CH1_scale, label='Spectre')
            plt.plot(self.tab_lambda, self.creneaux / self.CH2_scale, label='Créneau')
            plt.xlabel(r'$\lambda \;(nm)$')
            plt.ylabel(r'Signaux $(div)$')
            plt.legend()
            plt.savefig("Figure.png")

            print("Figure enregistrée dans Figure.png")
            MainWindow.setStatusTip("Figure enregistrée dans Figure.png")
        except:
            print('Aucune acquisition réalisée')
            MainWindow.setStatusTip('Aucune acquisition réalisée')
        
    def save_fig_as_jpg(self):
        try:
            figure = plt.figure()
            plt.plot(self.tab_lambda, self.spectre / self.CH1_scale, label='Spectre')
            plt.plot(self.tab_lambda, self.creneaux / self.CH2_scale, label='Créneau')
            plt.xlabel(r'$\lambda \;(nm)$')
            plt.ylabel(r'Signaux $(div)$')
            plt.legend()
            plt.savefig("Figure.jpg")

            print("Figure enregistrée dans Figure.jpg")
            MainWindow.setStatusTip("Figure enregistrée dans Figure.jpg")
        except:
            print('Aucune acquisition réalisée')
            MainWindow.setStatusTip('Aucune acquisition réalisée')
        
    
    

if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    app.exec_()

Aucune acquisition réalisée
Vous n'avez pas selectionné d'oscilloscope
Vous n'avez pas selectionné d'oscilloscope


C:\Users\Zakaria Bradai\AppData\Local\Temp\ipykernel_8776\1826521723.py:482: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  self.index_oscillo = int(np.where( np.array(self.answer_IDN)==text+'\n' )[0])


Pas d'appareil connecté, vérifier les paramètres entrés ou le branchement...


-----------------

-----------------

-----------------

# Version avec 2 curseurs :

In [1]:
from PyQt5 import QtCore, QtGui, QtWidgets

import pyvisa as visa
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import sys
from datetime import datetime
import time
from time import sleep
import pandas as pd

import serial
import serial.tools.list_ports

#------
# %matplotlib qt
#------
matplotlib.use('Qt5Agg')
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg, NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure
#---

def Select_central_value(tab):      
    '''Fonction qui sélectionne l'élément central d'un liste ou d'un array'''
    
    N = len(tab)
    if N%2==0:
        return tab[ int( N/2 - 1 )  ]
    else:
        return tab[ int ( (N-1)/2 - 1 ) ]
    
def Get_closest_value_index(tab,value):
    '''Fonction qui renvoie l'indice de l element de tab dont la valeur est la plus proche de value'''
    index = Select_central_value( np.where( np.abs(tab-value) == np.min( np.abs(tab-value) ) )[0] )
    return index

def Carac_Spectre(spectre, tab_lambda):
    '''Fonction qui renvoie la largeur a mi hauteur et la longueur d onde du pic d un spectre'''
    S_max = np.max(spectre)
    indice_max = Get_closest_value_index(spectre, S_max)
    
    # On divise le spectre en 2 regions, une à gauche du pic (S_inf) et l'autre à droite (S_sup)
    S_sup = spectre[indice_max:]
    S_inf = spectre[:indice_max+1]
    
    
    # On recupere l'indice de intensite la plus proches de 1/2 S_max pour les 2 signaux S_sup et S_inf
    indice1 = Get_closest_value_index(S_inf,0.5*S_max  )
    indice2 = Get_closest_value_index(S_sup,0.5*S_max  ) + len(S_inf)
    
    # On calcule la largeur du spectre à mi-hauteur
    largeur_mi_hauteur = tab_lambda[indice2] - tab_lambda[indice1]
    
    # On recupere la longueur d'onde du pic du spectre
    lambda_laser = tab_lambda[indice_max]
    
    return largeur_mi_hauteur,lambda_laser




#------------------------------------------------------------------------------------------------------------------------

def Calcul_lambda(creneau, t, t_c, lambda_c):
    ''' Fonction qui a servie à déterminer la valeur de a = d \lambda / d t'''
    tab_indices = np.where( (creneau>=0.8) & (creneau<2) )[0]
    tab_differences= tab_indices[1:] - tab_indices[:-1]
    plt.figure()#
    plt.plot(creneau[tab_indices])  #
    plt.show()
    tab_indices = tab_indices[1:]
    tab_indices_vrais = tab_indices[tab_differences==1]
    indice_etalon = Select_central_value( tab_indices_vrais )
    t_etalon = t[indice_etalon]
    
    a = 1 / np.abs(t_c - t_etalon)
    print(a)#
    
    tab_lambda = lambda_c + a * (t-t_c)
    return tab_lambda
    
#-----------------------------------------------------------------------------------------------


def Convert_lambda_t(t,t_c, lambda_c, t2):
    '''Fonction qui convertit l echelle horizontale en temps en longueur d onde'''
    
    a = 1 / np.abs(t_c - t2)
    print('a =',a)
    
    tab_lambda = lambda_c + a*t
    
    return tab_lambda

#-------------------------------------------------

class MplCanvas(FigureCanvasQTAgg):
    
    def __init__(self, parent=None, width=5, height=4, dpi=100):
        fig = Figure(figsize=(width, height), dpi=dpi)
        self.axes = fig.add_subplot(111)
        super(MplCanvas, self).__init__(fig)

#------------------


#------------------ Fenêtre Paramètres oscilloscope

class Ui_Dialog(object):
    def setupUi(self, Dialog):
        Dialog.setObjectName("Dialog")
        Dialog.resize(428, 358)
        self.splitter_11 = QtWidgets.QSplitter(Dialog)
        self.splitter_11.setGeometry(QtCore.QRect(30, 20, 371, 311))
        self.splitter_11.setOrientation(QtCore.Qt.Vertical)
        self.splitter_11.setObjectName("splitter_11")
        self.splitter_10 = QtWidgets.QSplitter(self.splitter_11)
        self.splitter_10.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_10.setObjectName("splitter_10")
        self.label_params_oscillo = QtWidgets.QLabel(self.splitter_10)
        self.label_params_oscillo.setObjectName("label_params_oscillo")
        self.pushButton = QtWidgets.QPushButton(self.splitter_10)
        self.pushButton.setObjectName("pushButton")
        self.splitter = QtWidgets.QSplitter(self.splitter_11)
        self.splitter.setOrientation(QtCore.Qt.Horizontal)
        self.splitter.setObjectName("splitter")
        self.label_bd_rate = QtWidgets.QLabel(self.splitter)
        self.label_bd_rate.setObjectName("label_bd_rate")
        self.lineEdit_baud_rate = QtWidgets.QLineEdit(self.splitter)
        self.lineEdit_baud_rate.setObjectName("lineEdit_baud_rate")
        self.splitter_9 = QtWidgets.QSplitter(self.splitter_11)
        self.splitter_9.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_9.setObjectName("splitter_9")
        self.label_hscale = QtWidgets.QLabel(self.splitter_9)
        self.label_hscale.setObjectName("label_hscale")
        self.lineEdit_hscale = QtWidgets.QLineEdit(self.splitter_9)
        self.lineEdit_hscale.setObjectName("lineEdit_hscale")
        self.splitter_2 = QtWidgets.QSplitter(self.splitter_11)
        self.splitter_2.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_2.setObjectName("splitter_2")
        self.label_hoffset = QtWidgets.QLabel(self.splitter_2)
        self.label_hoffset.setObjectName("label_hoffset")
        self.lineEdit_hoffset = QtWidgets.QLineEdit(self.splitter_2)
        self.lineEdit_hoffset.setObjectName("lineEdit_hoffset")
        self.splitter_8 = QtWidgets.QSplitter(self.splitter_11)
        self.splitter_8.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_8.setObjectName("splitter_8")
        self.label_vscaleCH1 = QtWidgets.QLabel(self.splitter_8)
        self.label_vscaleCH1.setObjectName("label_vscaleCH1")
        self.lineEdit_vscaleCH1 = QtWidgets.QLineEdit(self.splitter_8)
        self.lineEdit_vscaleCH1.setObjectName("lineEdit_vscaleCH1")
        self.splitter_7 = QtWidgets.QSplitter(self.splitter_11)
        self.splitter_7.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_7.setObjectName("splitter_7")
        self.label_vscaleCH2 = QtWidgets.QLabel(self.splitter_7)
        self.label_vscaleCH2.setObjectName("label_vscaleCH2")
        self.lineEdit_vscaleCH2 = QtWidgets.QLineEdit(self.splitter_7)
        self.lineEdit_vscaleCH2.setObjectName("lineEdit_vscaleCH2")
        self.splitter_6 = QtWidgets.QSplitter(self.splitter_11)
        self.splitter_6.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_6.setObjectName("splitter_6")
        self.labe_offsetCH1_V = QtWidgets.QLabel(self.splitter_6)
        self.labe_offsetCH1_V.setObjectName("labe_offsetCH1_V")
        self.lineEdit_offsetCH1_V = QtWidgets.QLineEdit(self.splitter_6)
        self.lineEdit_offsetCH1_V.setObjectName("lineEdit_offsetCH1_V")
        self.splitter_5 = QtWidgets.QSplitter(self.splitter_11)
        self.splitter_5.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_5.setObjectName("splitter_5")
        self.label_offsetCH2_V = QtWidgets.QLabel(self.splitter_5)
        self.label_offsetCH2_V.setObjectName("label_offsetCH2_V")
        self.lineEdit_offsetCH2_V = QtWidgets.QLineEdit(self.splitter_5)
        self.lineEdit_offsetCH2_V.setObjectName("lineEdit_offsetCH2_V")
        self.splitter_4 = QtWidgets.QSplitter(self.splitter_11)
        self.splitter_4.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_4.setObjectName("splitter_4")
        self.label_offsetCH1_V = QtWidgets.QLabel(self.splitter_4)
        self.label_offsetCH1_V.setObjectName("label_offsetCH1_V")
        self.lineEdit_offsetCH1_div = QtWidgets.QLineEdit(self.splitter_4)
        self.lineEdit_offsetCH1_div.setObjectName("lineEdit_offsetCH1_div")
        self.splitter_3 = QtWidgets.QSplitter(self.splitter_11)
        self.splitter_3.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_3.setObjectName("splitter_3")
        self.label_offsetCH2_div = QtWidgets.QLabel(self.splitter_3)
        self.label_offsetCH2_div.setObjectName("label_offsetCH2_div")
        self.lineEdit_offsetCH2_div = QtWidgets.QLineEdit(self.splitter_3)
        self.lineEdit_offsetCH2_div.setObjectName("lineEdit_offsetCH2_div")

        self.retranslateUi(Dialog)
        QtCore.QMetaObject.connectSlotsByName(Dialog)

    def retranslateUi(self, Dialog):
        _translate = QtCore.QCoreApplication.translate
        Dialog.setWindowTitle(_translate("Dialog", "Paramètres de l\'oscilloscope"))
        self.label_params_oscillo.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:12pt; font-weight:600; text-decoration: underline;\">Paramètres Oscilloscope :</span></p></body></html>"))
        self.pushButton.setText(_translate("Dialog", "Valider les paramètres"))
        self.label_bd_rate.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:10pt;\">Baud rate :</span></p></body></html>"))
        self.label_hscale.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:10pt;\">Horizontal Scale (en s) :</span></p></body></html>"))
        self.label_hoffset.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:10pt;\">Horizontal offset (en s) :</span></p></body></html>"))
        self.label_vscaleCH1.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:10pt;\">Vertical Scale CH1 (en V) :</span></p></body></html>"))
        self.label_vscaleCH2.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:10pt;\">Vertical Scale CH2 (en V) :</span></p></body></html>"))
        self.labe_offsetCH1_V.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:10pt;\">Offset CH1 (en V) :</span></p></body></html>"))
        self.label_offsetCH2_V.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:10pt;\">Offset CH2 (en V) :</span></p></body></html>"))
        self.label_offsetCH1_V.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:10pt;\">Offset CH1 (en div) :</span></p></body></html>"))
        self.label_offsetCH2_div.setText(_translate("Dialog", "<html><head/><body><p><span style=\" font-size:10pt;\">Offset CH2 (en div) :</span></p></body></html>"))


#---------------------


class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        
        
        
        #--------------------------
        
        self.rm = visa.ResourceManager()    # Appareils Pyvisa disponibles 
        self.serial_port = serial.tools.list_ports.comports()   # Ports série disponibles
        
        #-------------- MainWindow
        MainWindow.setObjectName("MainWindow")
        #MainWindow.resize(646, 626)
        MainWindow.resize(646, 275)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        
        MainWindow.setStatusTip("Bienvenue, veuillez sélectionner l'oscilloscope")
        
        #-------------------- Plot Widget
        self.widget = QtWidgets.QWidget(self.centralwidget)
        self.widget.setGeometry(QtCore.QRect(10, 230, 621, 351))
        self.widget.setObjectName("widget")
        self.widget.hide() ###---------------
        #-------------
        self.splitter_7 = QtWidgets.QSplitter(self.centralwidget)
        self.splitter_7.setGeometry(QtCore.QRect(10, 8, 621, 211))
        self.splitter_7.setOrientation(QtCore.Qt.Vertical)
        self.splitter_7.setObjectName("splitter_7")
        self.splitter = QtWidgets.QSplitter(self.splitter_7)
        self.splitter.setOrientation(QtCore.Qt.Horizontal)
        self.splitter.setObjectName("splitter")
        self.label_connexion_type = QtWidgets.QLabel(self.splitter)
        self.label_connexion_type.setObjectName("label_connexion_type")
        self.label_bdrate = QtWidgets.QLabel(self.splitter)
        self.label_bdrate.setObjectName("label_bdrate")
        self.lineEdit_bdrate = QtWidgets.QLineEdit(self.splitter)
        self.lineEdit_bdrate.setObjectName("lineEdit_bdrate")
        self.label_timeout = QtWidgets.QLabel(self.splitter)
        self.label_timeout.setObjectName("label_timeout")
        self.lineEdit_timeout = QtWidgets.QLineEdit(self.splitter)
        self.lineEdit_timeout.setObjectName("lineEdit_timeout")
        self.splitter_2 = QtWidgets.QSplitter(self.splitter_7)
        self.splitter_2.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_2.setObjectName("splitter_2")
        
        #---------------- Selection oscilloscope--------------
        self.label_select_oscillo = QtWidgets.QLabel(self.splitter_2)
        self.label_select_oscillo.setObjectName("label_select_oscillo")
        self.comboBox_select_oscillo = QtWidgets.QComboBox(self.splitter_2)
        self.comboBox_select_oscillo.setObjectName("comboBox_select_oscillo")
        
        self.answer_IDN = []
        self.comboBox_select_oscillo.addItem('IDN des appareils connectés')
        
        self.nb_devices_pyvisa = len( self.rm.list_resources() )
        
        # On liste les appareils connectés avec Pyvisa
        for i in range( self.nb_devices_pyvisa ):   
            ressource = self.rm.open_resource( self.rm.list_resources()[i] ) 
            self.answer_IDN.append( ressource.query('*IDN?',0.001) )
            self.comboBox_select_oscillo.addItem( self.answer_IDN[i].strip())  # .strip() enlève le '\n'
            #ressource.close()
        
        self.list_ressources = list(np.copy(self.answer_IDN))
        
        # On liste les appareils connectés avec Pyserial
        for port in self.serial_port:
            #com_port = f"{port.device}"
            com_port = port.device
            ressource = serial.Serial(com_port, 9600, timeout=1)
            ressource.write(b'*IDN? \n')
            sleep(1)
            IDN = ressource.readline().decode('ascii')
            if IDN =='':
                self.answer_IDN.append( f"{port}"+'\n' )    # On vérifie s'il s'agit bien d'un oscillo Tektronix
                self.comboBox_select_oscillo.addItem( f"{port}" )
                self.list_ressources.append(com_port)
            if IDN!='': 
                self.answer_IDN.append( IDN )
                self.comboBox_select_oscillo.addItem( IDN.strip() )
                self.list_ressources.append(com_port)
            ressource.close()
        
        #-----------Params-------
        self.button_params_oscillo = QtWidgets.QPushButton(self.splitter_7)
        self.button_params_oscillo.setObjectName("button_params_oscillo")
        self.splitter_3 = QtWidgets.QSplitter(self.splitter_7)
        self.splitter_3.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_3.setObjectName("splitter_3")
        self.label_lambda_c = QtWidgets.QLabel(self.splitter_3)
        self.label_lambda_c.setObjectName("label_lambda_c")
        self.spinBox_lambda_c = QtWidgets.QSpinBox(self.splitter_3)
        self.spinBox_lambda_c.setMinimum(350)
        self.spinBox_lambda_c.setMaximum(1100)
        self.spinBox_lambda_c.setObjectName("spinBox_lambda_c")
        self.splitter_4 = QtWidgets.QSplitter(self.splitter_7)
        self.splitter_4.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_4.setObjectName("splitter_4")
        self.label_vcursor = QtWidgets.QLabel(self.splitter_4)
        self.label_vcursor.setObjectName("label_vcursor")
        self.button_appear_vcursor = QtWidgets.QPushButton(self.splitter_4)
        self.button_appear_vcursor.setObjectName("button_appear_vcursor")
        self.checkBox_cursor = QtWidgets.QPushButton(self.splitter_4)
        self.checkBox_cursor.setObjectName("checkBox_cursor")
        self.button_acquire = QtWidgets.QPushButton(self.splitter_7)
        self.button_acquire.setObjectName("button_acquire")
        self.splitter_5 = QtWidgets.QSplitter(self.splitter_7)
        self.splitter_5.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_5.setObjectName("splitter_5")
        self.label_lambda_pic = QtWidgets.QLabel(self.splitter_5)
        self.label_lambda_pic.setObjectName("label_lambda_pic")
        self.value_lambda_pic = QtWidgets.QLineEdit(self.splitter_5)
        self.value_lambda_pic.setReadOnly(True)
        self.value_lambda_pic.setObjectName("value_lambda_pic")
        self.label_largeur = QtWidgets.QLabel(self.splitter_5)
        self.label_largeur.setObjectName("label_largeur")
        self.label_largeur_mi_hauteur = QtWidgets.QLineEdit(self.splitter_5)
        self.label_largeur_mi_hauteur.setReadOnly(True)
        self.label_largeur_mi_hauteur.setObjectName("label_largeur_mi_hauteur")
        self.splitter_6 = QtWidgets.QSplitter(self.splitter_7)
        self.splitter_6.setOrientation(QtCore.Qt.Horizontal)
        self.splitter_6.setObjectName("splitter_6")
        self.label_date_acquisition = QtWidgets.QLabel(self.splitter_6)
        self.label_date_acquisition.setObjectName("label_date_acquisition")
        self.label_date = QtWidgets.QLineEdit(self.splitter_6)
        self.label_date.setText("")
        self.label_date.setReadOnly(True)
        self.label_date.setObjectName("label_date")
        MainWindow.setCentralWidget(self.centralwidget)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)
        self.menuBar = QtWidgets.QMenuBar(MainWindow)
        self.menuBar.setGeometry(QtCore.QRect(0, 0, 646, 21))
        self.menuBar.setObjectName("menuBar")
        self.menuEnregister_l_acquisition_en = QtWidgets.QMenu(self.menuBar)
        self.menuEnregister_l_acquisition_en.setObjectName("menuEnregister_l_acquisition_en")
        #self.menuHelp = QtWidgets.QMenu(self.menuBar)
        #self.menuHelp.setObjectName("menuHelp")
        self.menuEnregistrer_l_acquisition_en = QtWidgets.QMenu(self.menuBar)
        self.menuEnregistrer_l_acquisition_en.setObjectName("menuEnregistrer_l_acquisition_en")
        MainWindow.setMenuBar(self.menuBar)
        self.action_png = QtWidgets.QAction(MainWindow)
        self.action_png.setObjectName("action_png")
        self.action_jpg = QtWidgets.QAction(MainWindow)
        self.action_jpg.setObjectName("action_jpg")
        self.action_csv = QtWidgets.QAction(MainWindow)
        self.action_csv.setObjectName("action_csv")
        self.action_txt = QtWidgets.QAction(MainWindow)
        self.action_txt.setObjectName("action_txt")
        self.action_png_2 = QtWidgets.QAction(MainWindow)
        self.action_png_2.setObjectName("action_png_2")
        self.action_jpg_2 = QtWidgets.QAction(MainWindow)
        self.action_jpg_2.setObjectName("action_jpg_2")
        self.action_csv_2 = QtWidgets.QAction(MainWindow)
        self.action_csv_2.setObjectName("action_csv_2")
        self.action_txt_2 = QtWidgets.QAction(MainWindow)
        self.action_txt_2.setObjectName("action_txt_2")
        self.action_png_3 = QtWidgets.QAction(MainWindow)
        self.action_png_3.setObjectName("action_png_3")
        self.action_jpg_3 = QtWidgets.QAction(MainWindow)
        self.action_jpg_3.setObjectName("action_jpg_3")
        self.action_csv_3 = QtWidgets.QAction(MainWindow)
        self.action_csv_3.setObjectName("action_csv_3")
        self.action_txt_3 = QtWidgets.QAction(MainWindow)
        self.action_txt_3.setObjectName("action_txt_3")
        self.actionGuide_d_utilisation = QtWidgets.QAction(MainWindow)
        self.actionGuide_d_utilisation.setObjectName("actionGuide_d_utilisation")
        self.action_csv_4 = QtWidgets.QAction(MainWindow)
        self.action_csv_4.setObjectName("action_csv_4")
        self.action_txt_4 = QtWidgets.QAction(MainWindow)
        self.action_txt_4.setObjectName("action_txt_4")
        self.menuEnregister_l_acquisition_en.addAction(self.action_png_3)
        self.menuEnregister_l_acquisition_en.addAction(self.action_jpg_3)
        #self.menuHelp.addAction(self.actionGuide_d_utilisation)
        self.menuEnregistrer_l_acquisition_en.addAction(self.action_csv_4)
        self.menuEnregistrer_l_acquisition_en.addAction(self.action_txt_4)
        self.menuBar.addAction(self.menuEnregistrer_l_acquisition_en.menuAction())
        self.menuBar.addAction(self.menuEnregister_l_acquisition_en.menuAction())
        #self.menuBar.addAction(self.menuHelp.menuAction())

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)
        
        
        # --------------- Figure-----------------------------------------------------------
    
    
    
        self.sc = MplCanvas(MainWindow, width=5, height=4, dpi=100)
        self.toolbar = NavigationToolbar(self.sc, MainWindow)
        self.layout = QtWidgets.QVBoxLayout()
        self.layout.addWidget(self.toolbar)
        self.layout.addWidget(self.sc)
        self.widget.setLayout(self.layout)
        
        
        #----- Fenêtre Paramètres Oscilloscope-----------------------------------------------------------
        self.Dialog = QtWidgets.QDialog()
        self.ui_Dialog = Ui_Dialog()
        self.ui_Dialog.setupUi(self.Dialog)
        self.ui_Dialog.lineEdit_offsetCH1_div.setReadOnly(True)
        self.ui_Dialog.lineEdit_offsetCH2_div.setReadOnly(True)

        # --------------------Actions--------------------
        
        # Selection de l'oscilloscope via la ComboBox
        self.comboBox_select_oscillo.activated[str].connect(self.select_ressource)
        
        # faire apparaître les curseurs verticaux
        self.button_appear_vcursor.clicked.connect(self.appear_vcursor)
        
        # Valider les positions des curseurs
        self.checkBox_cursor.clicked.connect(self.maj_cursors)
        
        # Acquisition
        self.button_acquire.clicked.connect(self.acquire_data)
        
        self.connect = None   # type de la connexion : pyvisa ou pyserial
        
        # Ouvrir la fenêtre paramètres de l'oscilloscope :
        self.button_params_oscillo.clicked.connect(self.open_params_window)
        
        self.ui_Dialog.pushButton.clicked.connect(self.update_params)
        
        
        
        #----------------------------------------- QActions
        
        self.action_csv_4.triggered.connect(self.save_acq_as_csv)
        self.action_txt_4.triggered.connect(self.save_acq_as_txt)
        
        self.action_png_3.triggered.connect(self.save_fig_as_png)
        self.action_jpg_3.triggered.connect(self.save_fig_as_jpg)
        
        
        
        
        
        
        #-----------------------------------------------------------

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.label_connexion_type.setText(_translate("MainWindow", "<html><head/><body><p><span style=\" font-size:10pt;\">Connexion avec pyserial :</span></p></body></html>"))
        self.label_bdrate.setText(_translate("MainWindow", "Baudrate"))
        self.label_timeout.setText(_translate("MainWindow", "Timeout"))
        self.label_select_oscillo.setText(_translate("MainWindow", "<html><head/><body><p><span style=\" font-size:10pt; font-weight:600;\">Selectionnez l\'oscilloscope :</span></p></body></html>"))
        self.button_params_oscillo.setText(_translate("MainWindow", "Paramètres de l\'oscilloscope"))
        self.label_lambda_c.setText(_translate("MainWindow", "<html><head/><body><p><span style=\" font-size:10pt;\">Veuillez entrer la longuer d\'onde associée au curseur de l\'analyseur de spectres (nm) :</span></p></body></html>"))
        self.label_vcursor.setText(_translate("MainWindow", "<html><head/><body><p><span style=\" font-size:9pt;\">Veuillez placer le curseur 1 de l\'oscilloscope sur le curseur de l\'analyseur :</span></p></body></html>"))
        self.button_appear_vcursor.setText(_translate("MainWindow", "Faire apparaître le curseur"))
        self.checkBox_cursor.setText(_translate("MainWindow", "Valider"))
        self.button_acquire.setText(_translate("MainWindow", "Lancer l\'acquisition"))
        self.label_lambda_pic.setText(_translate("MainWindow", "<html><head/><body><p><span style=\" font-size:10pt;\">Longueur d\'onde du pic :</span></p></body></html>"))
        self.label_largeur.setText(_translate("MainWindow", "<html><head/><body><p><span style=\" font-size:10pt;\">Largeur à mi-hauteur :</span></p></body></html>"))
        self.label_date_acquisition.setText(_translate("MainWindow", "<html><head/><body><p><span style=\" font-size:10pt;\">Date de l\'acquisition :</span></p></body></html>"))
        self.menuEnregister_l_acquisition_en.setTitle(_translate("MainWindow", "Enregister la figure en"))
        #self.menuHelp.setTitle(_translate("MainWindow", "Help"))
        self.menuEnregistrer_l_acquisition_en.setTitle(_translate("MainWindow", "Enregistrer l\'acquisition en"))
        self.action_png.setText(_translate("MainWindow", ".png"))
        self.action_jpg.setText(_translate("MainWindow", ".jpg"))
        self.action_csv.setText(_translate("MainWindow", ".csv"))
        self.action_txt.setText(_translate("MainWindow", ".txt"))
        self.action_png_2.setText(_translate("MainWindow", ".png"))
        self.action_jpg_2.setText(_translate("MainWindow", ".jpg"))
        self.action_csv_2.setText(_translate("MainWindow", ".csv"))
        self.action_txt_2.setText(_translate("MainWindow", ".txt"))
        self.action_png_3.setText(_translate("MainWindow", ".png"))
        self.action_jpg_3.setText(_translate("MainWindow", ".jpg"))
        self.action_csv_3.setText(_translate("MainWindow", ".csv"))
        self.action_txt_3.setText(_translate("MainWindow", ".txt"))
        self.actionGuide_d_utilisation.setText(_translate("MainWindow", "Guide d\'utilisation"))
        self.action_csv_4.setText(_translate("MainWindow", ".csv"))
        self.action_txt_4.setText(_translate("MainWindow", ".txt"))
        
    #   ----------------------------- Fonctions ----- Logique -----------------------------
    
        
    def select_ressource(self, text):    # Selection de l'appareil via la ComboBox
        try:
            self.index_oscillo = int(np.where( np.array(self.answer_IDN)==text+'\n' )[0])
            if self.index_oscillo < self.nb_devices_pyvisa:
                self.oscillo = self.rm.open_resource( self.rm.list_resources()[self.index_oscillo] )
                print("Appareil connecté avec Pyvisa")  ###
                print("IDN de le l'appareil sélectionné :"+self.oscillo.query('*IDN?',0.001)) ###
                
                MainWindow.setStatusTip("Connexion avec Pyvisa")
                
                self.connect = 'pyvisa'
                
                # UPDATE PARAMS OSCILLO
                
                self.ui_Dialog.lineEdit_baud_rate.setText(self.oscillo.query('RS232:BAUD?',0.01))
                echelle1 = float(self.oscillo.query('CH1:SCALE?',0.01))
                offset1_div = float(self.oscillo.query('CH1:POS?',0.01))
                offset1_V = offset1_div * echelle1
                self.ui_Dialog.lineEdit_vscaleCH1.setText(str(echelle1))
                self.ui_Dialog.lineEdit_offsetCH1_V.setText(str(offset1_V))
                self.ui_Dialog.lineEdit_offsetCH1_div.setText(str(offset1_div))
                echelle2 = float(self.oscillo.query('CH2:SCALE?',0.01))
                offset2_div = float(self.oscillo.query('CH2:POS?',0.01))
                offset2_V = offset2_div * echelle2
                self.ui_Dialog.lineEdit_vscaleCH2.setText(str(echelle2))
                self.ui_Dialog.lineEdit_offsetCH2_V.setText(str(offset2_V))
                self.ui_Dialog.lineEdit_offsetCH2_div.setText(str(offset2_div))
                self.ui_Dialog.lineEdit_hscale.setText(self.oscillo.query('HORIZONTAL:SCALE?',0.01))
                self.ui_Dialog.lineEdit_hoffset.setText(self.oscillo.query('HORIZONTAL:DELAY:POS?',0.01))
                
                
            else:
                self.bd_rate = int(self.lineEdit_bdrate.text())
                self.timeout = int(self.lineEdit_timeout.text())
                self.com = self.list_ressources[self.index_oscillo]
                self.oscillo = serial.Serial(self.com, self.bd_rate, timeout=self.timeout)
                print("Appareil connecté avec Pyserial")   ###
                print("IDN de le l'appareil sélectionné :"+self.answer_IDN[self.index_oscillo]) ###
                
                MainWindow.setStatusTip("Connexion avec Pyserial "+self.com)
                
                self.connect = 'pyserial'
                
                #---- M A J params oscilloscope
                self.oscillo.write(b'RS232:BAUD? \n')
                self.ui_Dialog.lineEdit_baud_rate.setText(self.oscillo.readline().decode('ascii'))
                
                self.oscillo.write(b'CH1:SCALE? \n')
                echelle1 = float(self.oscillo.readline().decode('ascii'))
                self.oscillo.write(b'CH1:POS? \n')
                offset1_div = float(self.oscillo.readline().decode('ascii'))
                offset1_V = offset1_div * echelle1
                self.ui_Dialog.lineEdit_vscaleCH1.setText(str(echelle1))
                self.ui_Dialog.lineEdit_offsetCH1_V.setText(str(offset1_V))
                self.ui_Dialog.lineEdit_offsetCH1_div.setText(str(offset1_div))
                
                self.oscillo.write(b'CH2:SCALE? \n')
                echelle2 = float(self.oscillo.readline().decode('ascii'))
                self.oscillo.write(b'CH2:POS? \n')
                offset2_div = float(self.oscillo.readline().decode('ascii'))
                offset2_V = offset2_div * echelle2
                self.ui_Dialog.lineEdit_vscaleCH2.setText(str(echelle2))
                self.ui_Dialog.lineEdit_offsetCH2_V.setText(str(offset2_V))
                self.ui_Dialog.lineEdit_offsetCH2_div.setText(str(offset2_div))
                
                self.oscillo.write(b'HORIZONTAL:SCALE? \n')
                self.ui_Dialog.lineEdit_hscale.setText(self.oscillo.readline().decode('ascii'))
                
                self.oscillo.write(b'HORIZONTAL:DELAY:POS? \n')
                self.ui_Dialog.lineEdit_hoffset.setText(self.oscillo.readline().decode('ascii'))
                
        except:
            print("Pas d'appareil connecté, vérifier les paramètres entrés...")
            MainWindow.setStatusTip("Pas d'appareil connecté, vérifier les paramètres entrés ou le branchement...")
            
    #---------
        
    def appear_vcursor(self):
        if self.connect == "pyvisa":
            self.oscillo.write('CURSOR:FUNC VBA')
        elif self.connect == "pyserial":
            self.oscillo.write(b'CURSOR:FUNC VBA \n')
        else:
            print("Vous n'avez pas selectionné d'oscilloscope")
            MainWindow.setStatusTip("Vous n'avez pas selectionné d'oscilloscope")
        
    #---------
        
        
    def maj_cursors(self):
        self.lambda_c = self.spinBox_lambda_c.value()
        if self.connect == 'pyvisa':
            self.t_c_read = float( self.oscillo.query('CURSOR:VBARS:POSITION1?') )
            self.t2 = float( self.oscillo.query('CURSOR:VBARS:POSITION2?') )
            self.oscillo.write('CURSOR:FUNC OFF')
            print("Longueur d'onde du curseur : "+str(self.lambda_c)+' nm')
            MainWindow.setStatusTip("Longueur d'onde du curseur : "+str(self.lambda_c)+' nm')
        elif self.connect == 'pyserial':
            self.oscillo.write(b'CURSOR:VBARS:POSITION1? \n')
            self.t_c_read = float( self.oscillo.readline().decode('ascii') )
            self.oscillo.write(b'CURSOR:VBARS:POSITION2? \n')
            self.t2 = float( self.oscillo.readline().decode('ascii') )
            self.oscillo.write(b'CURSOR:FUNC OFF \n')
            print("Longueur d'onde du curseur : "+str(self.lambda_c)+' nm')
            MainWindow.setStatusTip("Longueur d'onde du curseur : "+str(self.lambda_c)+' nm')
        else:
            print("Vous n'avez pas selectionné d'oscilloscope")
            MainWindow.setStatusTip("Vous n'avez pas selectionné d'oscilloscope")
        
    #---------
        
        
    def acquire_data(self):
        
        try:
            if self.connect=='pyvisa':
                self.oscillo.write('DATA:SOURCE CH1')  # Select CH1
                self.coef_spectre = float(self.oscillo.query('WFMP:YMULT?',0.001))
                self.date_acquisition = datetime.now()
                self.spectre = np.fromstring(self.oscillo.query('CURVE?',0.001), sep=',') * self.coef_spectre

                self.oscillo.write('DATA:SOURCE CH2')
                self.coef_creneau = float(self.oscillo.query('WFMP:YMULT?',0.001))
                self.creneaux = np.fromstring(self.oscillo.query('CURVE?',0.001), sep=',') * self.coef_creneau

                self.t0 = float(self.oscillo.query('WFMP:XZERO?'))  # temps initial
                self.dt = float(self.oscillo.query('WFMP:XIN?'))   # intervalle d'échantillionage
                
                self.CH1_scale = float(self.oscillo.query('CH1:SCALE?',0.001))
                self.CH2_scale = float(self.oscillo.query('CH2:SCALE?',0.001))

                self.offset_spectre = float(self.oscillo.query('CH1:POS?',0.001)) * self.CH1_scale
                self.offset_creneau = float(self.oscillo.query('CH2:POS?',0.001)) * self.CH2_scale

            elif self.connect=='pyserial':
                
                self.oscillo.write(b'WFMP:XZERO? \n')
                self.t0 = float(self.oscillo.readline().decode('ascii'))

                self.oscillo.write(b'WFMP:XIN? \n')
                self.dt = float(self.oscillo.readline().decode('ascii'))

                self.oscillo.write(b'CH1:POS? \n')
                offset_spectre = float(self.oscillo.readline().decode('ascii'))
                self.oscillo.write(b'CH2:POS? \n')
                offset_creneau = float(self.oscillo.readline().decode('ascii'))

                self.oscillo.write(b'CH1:SCALE? \n')
                self.CH1_scale = float(self.oscillo.readline().decode('ascii'))
                self.offset_spectre = offset_spectre * self.CH1_scale
                self.oscillo.write(b'CH2:SCALE? \n')
                self.CH2_scale = float(self.oscillo.readline().decode('ascii'))
                self.offset_creneau = offset_creneau * self.CH2_scale

                self.oscillo.write(b'DATA:SOURCE CH1 \n')  # Select CH1

                self.date_acquisition = datetime.now()
                self.oscillo.write(b'CURVE? \n')
                sleep(1)
                V1 = np.fromstring(self.oscillo.readline().decode('ascii'), sep=',')
                self.oscillo.write(b'WFMP:YMULT? \n')
                self.coef_spectre = float(self.oscillo.readline().decode('ascii'))

                self.spectre = V1 * self.coef_spectre

                self.oscillo.write(b'DATA:SOURCE CH2 \n')
                self.oscillo.write(b'WFMP:YMULT? \n')
                self.coef_creneau = float(self.oscillo.readline().decode('ascii'))
                self.oscillo.write(b'CURVE? \n')
                sleep(1)
                V2 = np.fromstring(self.oscillo.readline().decode('ascii'), sep=',')

                self.creneaux = V2 * self.coef_creneau
                
                
                #----------------------------------------------


            self.spectre_calcul = self.spectre - self.offset_spectre
            self.creneaux_calcul = self.creneaux - self.offset_creneau   # PAS UTILE

            self.t = np.linspace(self.t0, self.t0 + 2500*self.dt,2500, endpoint=True)

            self.indice_curseur = Get_closest_value_index(self.t,self.t_c_read)

            self.t_c = self.t[self.indice_curseur]

            
            # Conversion temps---longueur d'onde----------------------------------------
            
            
            self.tab_lambda = Convert_lambda_t(self.t,self.t_c, self.lambda_c, self.t2)

                        
            #---------------------------------------------------------------------------

            
            self.largeur_mi_hauteur, self.lambda_pic = Carac_Spectre(self.spectre_calcul, self.tab_lambda)

            self.value_lambda_pic.setText('%.1f nm' %self.lambda_pic)
            self.label_largeur_mi_hauteur.setText('%.1f nm' %self.largeur_mi_hauteur)
            date_mesure = self.date_acquisition.strftime('%Y-%m-%d') + '                             ' + self.date_acquisition.strftime('%H') + 'h : '+ self.date_acquisition.strftime('%M') + 'min : ' + self.date_acquisition.strftime('%S') + 's'
            self.label_date.setText(date_mesure) 

            # Mise-à-jour du plot
            self.layout.removeWidget(self.sc)
            self.layout.removeWidget(self.toolbar)
            self.sc = MplCanvas(MainWindow, width=5, height=3)  # , -dpi=100)
            self.toolbar = NavigationToolbar(self.sc, MainWindow)
            coef_affichage = np.max(self.spectre) / np.max(self.creneaux)
            self.sc.axes.plot(self.tab_lambda, self.creneaux / self.CH2_scale ,alpha=0.5)
            self.sc.axes.plot(self.tab_lambda, self.spectre / self.CH1_scale ,label='Spectre',color='r')
            self.sc.axes.set_xlabel(r'Longueur d onde $\lambda \; \; $(nm)')
            self.sc.axes.set_ylabel(r'Signal (V)')
            self.sc.axes.legend()
            self.layout.addWidget(self.toolbar)
            self.layout.addWidget(self.sc)
            
            #self.widget.setLayout(layout)
            
            print('Acquisition réussie avec succès')
            MainWindow.setStatusTip("Acquisition réussie avec succès")
            
            MainWindow.resize(646, 626)
            self.widget.show()
        
        except:
            print("Vous n'avez pas selectionné d'oscilloscope")
            MainWindow.setStatusTip("Vous n'avez pas selectionné d'oscilloscope")
            
            
    def open_params_window(self):
        self.Dialog.show()
        
        
    def update_params(self):
        try:

            if self.connect=='pyserial':

                baud_rate = int(self.ui_Dialog.lineEdit_baud_rate.text())
                self.oscillo.write(b'RS232:BAUD %i \n'%baud_rate)

                hscale = float(self.ui_Dialog.lineEdit_hscale.text())
                self.oscillo.write(b'HORIZONTAL:SCALE %f \n'%hscale)

                hoffset = float(self.ui_Dialog.lineEdit_hoffset.text())
                self.oscillo.write(b'HORIZONTAL:POS %f \n'%hoffset)

                vscale1 = float(self.ui_Dialog.lineEdit_vscaleCH1.text())
                self.oscillo.write(b'CH1:SCALE %f \n'%vscale1)

                vscale2 = float(self.ui_Dialog.lineEdit_vscaleCH2.text())
                self.oscillo.write(b'CH2:SCALE %f \n'%vscale2)

                echelle1 = float(self.ui_Dialog.lineEdit_offsetCH1_V.text())
                voffset1 = echelle1/vscale1
                self.oscillo.write(b'CH1:POS %f \n'%voffset1)

                echelle2 = float(self.ui_Dialog.lineEdit_offsetCH2_V.text())
                voffset2 = echelle2/vscale2
                self.oscillo.write(b'CH2:POS %f \n'%voffset2)

                sleep(1)
                # Mise à jour de l'affichage :

                self.oscillo.write(b'RS232:BAUD?\n')
                sleep(0.2)
                self.ui_Dialog.lineEdit_baud_rate.setText(self.oscillo.readline().decode('ascii'))   
                self.oscillo.write(b'CH1:SCALE? \n')
                sleep(0.1)
                echelle1 = float(self.oscillo.readline().decode('ascii'))
                self.oscillo.write(b'CH1:POS? \n')
                sleep(0.1)
                offset1_div = float(self.oscillo.readline().decode('ascii'))
                offset1_V = offset1_div * echelle1
                self.ui_Dialog.lineEdit_vscaleCH1.setText(str(echelle1))
                self.ui_Dialog.lineEdit_offsetCH1_V.setText(str(offset1_V))
                self.ui_Dialog.lineEdit_offsetCH1_div.setText(str(offset1_div))   
                self.oscillo.write(b'CH2:SCALE? \n')
                sleep(0.1)
                echelle2 = float(self.oscillo.readline().decode('ascii'))
                self.oscillo.write(b'CH2:POS? \n')
                sleep(0.1)
                offset2_div = float(self.oscillo.readline().decode('ascii'))
                offset2_V = offset2_div * echelle2
                self.ui_Dialog.lineEdit_vscaleCH2.setText(str(echelle2))
                self.ui_Dialog.lineEdit_offsetCH2_V.setText(str(offset2_V))
                self.ui_Dialog.lineEdit_offsetCH2_div.setText(str(offset2_div))      
                self.oscillo.write(b'HORIZONTAL:SCALE? \n')
                sleep(0.1)
                self.ui_Dialog.lineEdit_hscale.setText(self.oscillo.readline().decode('ascii'))     
                self.oscillo.write(b'HORIZONTAL:POS? \n')
                sleep(0.1)
                self.ui_Dialog.lineEdit_hoffset.setText(self.oscillo.readline().decode('ascii'))

                print("Paramètres de l'oscilloscope mis-à-jour")
                MainWindow.setStatusTip("Paramètres de l'oscilloscope mis-à-jour")

            elif self.connect=='pyvisa':

                baud_rate = int(self.ui_Dialog.lineEdit_baud_rate.text())
                self.oscillo.write('RS232:BAUD %i'%baud_rate)

                hscale = float(self.ui_Dialog.lineEdit_hscale.text())
                self.oscillo.write('HORIZONTAL:SCALE %f'%hscale)

                hoffset = float(self.ui_Dialog.lineEdit_hoffset.text())
                self.oscillo.write('HORIZONTAL:POS %f'%hoffset)

                vscale1 = float(self.ui_Dialog.lineEdit_vscaleCH1.text())
                self.oscillo.write('CH1:SCALE %f'%vscale1)

                vscale2 = float(self.ui_Dialog.lineEdit_vscaleCH2.text())
                self.oscillo.write('CH2:SCALE %f'%vscale2)

                echelle1 = float(self.ui_Dialog.lineEdit_offsetCH1_V.text())
                voffset1 = echelle1/vscale1
                self.oscillo.write('CH1:POS %f'%voffset1)

                echelle2 = float(self.ui_Dialog.lineEdit_offsetCH2_V.text())
                voffset2 = echelle2/vscale2
                self.oscillo.write('CH2:POS %f'%voffset2)

                sleep(1)
                # Mise à jour de l'affichage :

                self.ui_Dialog.lineEdit_baud_rate.setText(self.oscillo.query('RS232:BAUD?',0.01)) 
                sleep(0.1)
                echelle1 = float(self.oscillo.query('CH1:SCALE?',0.01))
                offset1_div = float(self.oscillo.query('CH1:POS?',0.01))  
                offset1_V = offset1_div * echelle1
                self.ui_Dialog.lineEdit_vscaleCH1.setText(str(echelle1))
                self.ui_Dialog.lineEdit_offsetCH1_V.setText(str(offset1_V))
                self.ui_Dialog.lineEdit_offsetCH1_div.setText(str(offset1_div))   

                sleep(0.1)
                echelle2 = float(self.oscillo.query('CH2:SCALE?',0.01))
                offset2_div = float(self.oscillo.query('CH2:POS?',0.01))  
                offset2_V = offset2_div * echelle2
                self.ui_Dialog.lineEdit_vscaleCH2.setText(str(echelle2))
                self.ui_Dialog.lineEdit_offsetCH2_V.setText(str(offset2_V))
                self.ui_Dialog.lineEdit_offsetCH2_div.setText(str(offset2_div)) 

                sleep(0.1)
                self.ui_Dialog.lineEdit_hscale.setText(self.oscillo.query('HORIZONTAL:SCALE?',0.01))     
                sleep(0.1)
                self.ui_Dialog.lineEdit_hoffset.setText(self.oscillo.query('HORIZONTAL:POS?',0.01))

                print("Paramètres de l'oscilloscope mis-à-jour")
                MainWindow.setStatusTip("Paramètres de l'oscilloscope mis-à-jour")

            else:
                print("Vous n'avez pas selectionné d'oscilloscope")
                MainWindow.setStatusTip("Vous n'avez pas selectionné d'oscilloscope")
        except:
            pass
        
        
    def save_acq_as_txt(self):
        try:
            N = self.tab_lambda.size
            tab_data = np.zeros(N, dtype=[('lambda (nm)',float),('spectre (V)',float),('creneau',float)])
            tab_data['lambda (nm)'] = self.tab_lambda
            tab_data['spectre (V)'] = self.spectre
            tab_data['creneau'] = self.creneaux

            np.savetxt('Acquisition.txt',tab_data, delimiter=',', fmt='%f')

            print("Acquisition enregistrée dans Acquisition.txt")
            MainWindow.setStatusTip("Acquisition enregistrée dans Acquisition.txt")
        except:
            print('Aucune acquisition réalisée')
            MainWindow.setStatusTip('Aucune acquisition réalisée')
        
    def save_acq_as_csv(self):
        try:
            tab_CH1_scale = self.CH1_scale * np.ones(len(self.spectre))
            tab_CH2_scale = self.CH2_scale * np.ones(len(self.spectre))
            dict_data = {"Longueur d'onde (nm)":self.tab_lambda, "Spectre (V)":self.spectre, "Créneau (V)":self.creneaux,"CH1 Scale (Volt/div)":tab_CH1_scale,"CH2 Scale (Volt/div)":tab_CH2_scale}
            data_frame = pd.DataFrame(data=dict_data)
            data_frame.to_csv("Acquisition.csv")

            print("Acquisition enregistrée dans Acquisition.csv")
            MainWindow.setStatusTip("Acquisition enregistrée dans Acquisition.csv")
        except:
            print('Aucune acquisition réalisée')
            MainWindow.setStatusTip('Aucune acquisition réalisée')
        
    def save_fig_as_png(self):
        try:
            figure = plt.figure()
            plt.plot(self.tab_lambda, self.spectre / self.CH1_scale, label='Spectre')
            plt.plot(self.tab_lambda, self.creneaux / self.CH2_scale, label='Créneau')
            plt.xlabel(r'$\lambda \;(nm)$')
            plt.ylabel(r'Signaux $(div)$')
            plt.legend()
            plt.savefig("Figure.png")

            print("Figure enregistrée dans Figure.png")
            MainWindow.setStatusTip("Figure enregistrée dans Figure.png")
        except:
            print('Aucune acquisition réalisée')
            MainWindow.setStatusTip('Aucune acquisition réalisée')
        
    def save_fig_as_jpg(self):
        try:
            figure = plt.figure()
            plt.plot(self.tab_lambda, self.spectre / self.CH1_scale, label='Spectre')
            plt.plot(self.tab_lambda, self.creneaux / self.CH2_scale, label='Créneau')
            plt.xlabel(r'$\lambda \;(nm)$')
            plt.ylabel(r'Signaux $(div)$')
            plt.legend()
            plt.savefig("Figure.jpg")

            print("Figure enregistrée dans Figure.jpg")
            MainWindow.setStatusTip("Figure enregistrée dans Figure.jpg")
        except:
            print('Aucune acquisition réalisée')
            MainWindow.setStatusTip('Aucune acquisition réalisée')
        
    
    

if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    app.exec_()

Aucune acquisition réalisée
Aucune acquisition réalisée
Vous n'avez pas selectionné d'oscilloscope
Vous n'avez pas selectionné d'oscilloscope
Vous n'avez pas selectionné d'oscilloscope
Vous n'avez pas selectionné d'oscilloscope
